## Exploratory Data Analysis and Linear Models

### Importing the packages and setting the plot defaults

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # data plotting
import seaborn as sns # data visualisation and plotting
import statsmodels.api as sm # statistical modelling package
import statsmodels.formula.api as smf # statistical modelling package with R-like formulas
import scipy.stats as stats
import math

#sns.set(color_codes=True)

from statsmodels.genmod.generalized_linear_model import GLM # importing packages to run GLM
from statsmodels.genmod import families # importing families for exponential families
from scipy.stats import binom # to illustrate the binomial distribution.
from sklearn import datasets, linear_model # fetching iris dataset and linear model functions
from sklearn.metrics import mean_squared_error, r2_score

# Seaborn plot default configurations
sns.set_style("white")

# set the custom size for my graphs
sns.set(rc={'figure.figsize':(8.7,6.27)})


### Reading in the iris data

In [ ]:
# Define sklearn_to_df function to convert from sklearn to a pandas dataframes

def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Categorical.from_codes(sklearn_dataset.target, sklearn_dataset.target_names)
    return df

In [ ]:
# import and convert format of iris data to pandas data frame from sklearn
df_iris = sklearn_to_df(datasets.load_iris())

### Exercise 1

Run the following lines of code to explore the data

1. What are the dimensions of the dataframe?

In [ ]:
print(df_iris.shape)

2. What are the first and last values of sepal.length?

In [ ]:
df_iris.head(5)

df_iris.tail(5)

3. Which variables are float64s or objects?

In [ ]:
df_iris.info()

 4. Using the data summary, what is the minimum and maximum sepal length?

In [ ]:
df_iris.describe()

5. What are the names of the columns?

In [ ]:
# Let's simplify the column names and make them more meaningful

df_iris = df_iris.rename(columns = {'sepal length (cm)': 'sepal_length', 'sepal width (cm)': 'sepal_width', 'petal length (cm)': 'petal_length', 'petal width (cm)': 'petal_width','target': 'species'})

# What are the names of the columns?

df_iris.columns

### Visualising Distributions

How we visualise distributions depends on if the variable is

1. Categorical
2. Continuous

**Categorical Variable**: Frequency Plot of Species

In [ ]:
species_counts = sns.countplot(x= "species", data = df_iris)
species_counts

**Continuous Variable**: Distribution Plot of Petal Length

In [ ]:
petal_length_all_distplot = sns.distplot(df_iris['petal_length'], 
    hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3})

petal_length_all_distplot.set(xlabel='Petal_length', ylabel='Density')

petal_length_all_distplot

**Continuous Variable**: Distribution Plot of Petal Length by Species

In [ ]:
df_setosa = df_iris[df_iris.species == 'setosa']
petal_length_species = sns.distplot(df_setosa[['petal_length']], label = 'setosa', 
              hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3})

df_virginica = df_iris[df_iris.species == 'virginica']
petal_length_species = sns.distplot(df_virginica[['petal_length']], label = 'virginica', 
              hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3})

df_versicolor = df_iris[df_iris.species == 'versicolor']
petal_length_species = sns.distplot(df_versicolor[['petal_length']], label = 'versicolor', 
              hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3})


petal_length_species.set(xlabel='Petal Length', ylabel='Density x 10')
petal_length_species

#### Covariation

**Continuous vs. Categorical Variable**: Box plot of petal width by species

In [ ]:
petal_width_boxplot = sns.boxplot(data = df_iris, y = 'petal_width', x = 'species')
petal_width_boxplot

**Continuous vs. Categorical Variable**: Violin plot of sepal length for each species

In [ ]:
sepal_length_violin = sns.violinplot(data = df_iris, y = "sepal_length", x = 'species')
sepal_length_violin

**Continuous vs. Continuous Variable**: Scatterplot of petal width and petal length

In [ ]:
iris_scatter_petal_length_sepal_width = sns.scatterplot(data = df_iris, x = 'petal_length', 
                                                        y = 'sepal_width', hue = 'species')
iris_scatter_petal_length_sepal_width

### Exercises

1. Make a box plot or a violin plot of sepal width by species. How does this box plot/violin plot compare to the earlier box plot/violin plot we made of petal width and sepal length?

2. Make a scatterplot to visualise the relationship between petal length and sepal length coloured by species. What patterns can you pick out from the data?

3. Pairplots can be a quick and useful way to summarise your dataset quickly and to inspect the relationships simultaneously.Trying running the following code to make a pairplot. What does this code do?

`iris_all_pairplot = sns.pairplot(data = df_iris, hue="species", diag_kind="kde")
iris_all_pairplot`


In [ ]:
## Exercise 1

In [ ]:
## Exercise 2

In [ ]:
## Exercise 3

## Model Construction

### Model 1: Continuous Variable

We're interested in explaining the variation in `petal_width` using the explantory variable, `petal_length`.

Note that the smf.ols function automatically includes the intercept in the model, so we don't have to specify one.

In [ ]:
model1 = smf.ols(formula = 'petal_width ~ petal_length', data = df_iris)

results_mod1 = model1.fit()

print(results_mod1.summary())

In [ ]:
## Extracting variables from the model

# Intercept
print("Intercept = ",results_mod1.params['Intercept'])
# Slope
print("(Petal length) coef. = ", results_mod1.params['petal_length'])
# R-squared
print("R^2 = ", results_mod1.rsquared_adj)

Constructing the line of best fit:

In [ ]:
## Scatterplot of line of best fit compared to our previous guesses for
## the different slope and intercept to describe relationship between petal width and petal length}

iris_scatter = sns.scatterplot(data = df_iris, x = 'petal_length', y = 'petal_width', hue = 'species')

x = np.arange(7)
a = 0.37
a1 = 0.35

b = -0.2
b1 = -0.3

y = a*x + b
y1 = a1*x + b
y2 = results_mod1.params['petal_length']*x + results_mod1.params['Intercept']

iris_scatter = sns.regplot(x = x, y = y, marker="+")
iris_scatter = sns.regplot(x = x, y = y1, marker="+", line_kws = {"color": "red"})
iris_scatter = sns.regplot(x = x, y = y2, marker = "+", line_kws = {"color": "blue"})
iris_scatter


In [ ]:
## Line of best fit using regplot


iris_best_fit = sns.regplot(x = 'petal_length', y = 'petal_width', ci = 95, data = df_iris)
iris_best_fit

In [ ]:
## Inspecting the model residuals

df_iris_resid = sns.residplot(y = 'petal_width', x = 'petal_length', data = df_iris)
df_iris_resid


In [ ]:
## Inspecting the model residuals using a qqplot

resid = results_mod1.resid

# Use statsmodels qqplot to graph residuals
# make a figure and an axis
f, ax = plt.subplots(figsize=(7,7))
# call the qqplot graph from statsmodels 'graphics' module.
# fits against the normal distribution as standard.
sm.graphics.qqplot(resid, line='45', fit=True, ax=ax)

### Model 2: Continuous and Categorical Variables


In [ ]:
model2 = smf.ols(formula = 'petal_width ~ petal_length + C(species)', data = df_iris)

results_mod2 = model2.fit()

print(results_mod2.summary())

In [ ]:
## Line of best fit

iris_scatter_species = sns.scatterplot(data = df_iris, x = 'petal_length', y = 'petal_width', hue = 'species')

x = np.arange(7)
a = results_mod2.params['petal_length']

b_setosa = results_mod2.params['Intercept']
b_virginica = results_mod2.params['C(species)[T.virginica]'] + results_mod2.params['Intercept']
b_versicolor = results_mod2.params['C(species)[T.versicolor]']  + results_mod2.params['Intercept']


y_setosa = a*x + b_setosa
y_virginica = a*x + b_virginica
y_versicolor = a*x + b_versicolor

iris_scatter_species = sns.regplot(x = x, y = y_setosa, marker="+", line_kws = {"color": "blue"})
iris_scatter_species = sns.regplot(x = x, y = y_virginica, marker = "+", line_kws = {"color": "green"})
iris_scatter_species = sns.regplot(x = x, y = y_versicolor, marker = "+", line_kws = {"color": "orange"})

iris_scatter_species


In [ ]:
## Residuals for model 2

resid2 = results_mod2.resid
fitted2 = results_mod2.fittedvalues

resid2_plot = sns.scatterplot(x = fitted2, y = resid2)
resid2_plot.set(xlabel='fitted', ylabel='residuals')
resid2_plot

In [ ]:
## qqplot for model 2

# Use statsmodels qqplot to graph errors
# make a figure and an axis
f, ax = plt.subplots(figsize=(7,7))
# call the qqplot graph from statsmodels 'graphics' module.
# fits against the normal distribution as standard.

sm.graphics.qqplot(resid2, line='45', fit=True, ax=ax);

### Model 3: Interactions

In [ ]:
## Model Specification and Model Fit

model3 = smf.ols(formula = 'petal_width ~ petal_length*C(species)', data = df_iris)

results_mod3 = model3.fit()

print(results_mod3.summary())

In [ ]:
## Line of Best Fit

iris_scatter_species = sns.scatterplot(data = df_iris, x = 'petal_length', y = 'petal_width', hue = 'species')

x = np.arange(7)
a_setosa = results_mod3.params['petal_length']
a_virginica = results_mod3.params['petal_length:C(species)[T.virginica]'] + results_mod3.params['petal_length']
a_versicolor = results_mod3.params['petal_length:C(species)[T.versicolor]'] + results_mod3.params['petal_length']

b_setosa = results_mod3.params['Intercept']
b_virginica = results_mod3.params['C(species)[T.virginica]'] + results_mod3.params['Intercept']
b_versicolor = results_mod3.params['C(species)[T.versicolor]'] + results_mod3.params['Intercept']


y_setosa = a_setosa*x + b_setosa
y_virginica = a_virginica*x + b_virginica
y_versicolor = a_versicolor*x + b_versicolor

iris_scatter_species = sns.regplot(x = x, y = y_setosa, marker= "+", line_kws = {"color": "blue"})
iris_scatter_species = sns.regplot(x = x, y = y_virginica, marker = "+", line_kws = {"color": "green"})
iris_scatter_species = sns.regplot(x = x, y = y_versicolor, marker = "+", line_kws = {"color": "orange"})

iris_scatter_species


In [ ]:
## Residuals for Model 3

resid3 = results_mod3.resid
fitted3 = results_mod3.fittedvalues

resid3_plot = sns.scatterplot(x = fitted3, y = resid3)
resid3_plot


In [ ]:
## qqplot for Model 3

# Use statsmodels qqplot to graph errors
# make a figure and an axis
f, ax = plt.subplots(figsize=(7,7))
# call the qqplot graph from statsmodels 'graphics' module.
# fits against the normal distribution as standard.

sm.graphics.qqplot(resid3, line='45', fit=True, ax=ax);

### Model Comparison Exercise

**Exercises**

1. Let's compare the models we ran using Adjusted R^2 and AIC. Using the notes above, discuss in groups which model you think is best and why?

2. Take a look at the main parameters of Model 2 and Model 3 from the model summary tables. Do they seem to vary much between the models?



In [ ]:
## Exercise: Comparing R-squared
#Note, \n will put the results on the next line!

print("Adjusted R^2 Model 1 = ", results_mod1.rsquared_adj, "\nAdjusted R^2 Model 2 = ", results_mod2.rsquared_adj
, "\nAdjusted R^2 Model 3 = ", results_mod3.rsquared_adj)


## Comparing AIC

print("AIC Model 1 = ", results_mod1.aic, "\nAIC Model 2 = ", results_mod2.aic, "\nAIC Model 3 = ", results_mod3.aic)



In [ ]:
## Exercise: Take a look at the main parameters from Model 2 and Model 3 from the model summary tables

## Generalized Linear Models

The traditional statistical approach was to assume that all variation in the data was normally distributed, or to transform the data until it was, and then use classical methods based on the normal distribution to draw conclusions. 

In generalized linear models, variability isn't just a nuisance, but actually tells us something about the processes we are interested in. What we treat as "signal" and what we treat as "noise"  depends on our question. The same source of variability might provide an interesting insight into the data or be something we wish to account for so we can explore it further. 

In the next section, we will introduce you to a few common distributions that capture different forms of variation in the response variable from the exponential family.

We will then give practical examples of how to fit and interpret these models using python.

In [ ]:
## Normal Distribution

mu = 0
variance = 1
sigma = math.sqrt(variance)
norm_1 = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(norm_1, stats.norm.pdf(norm_1, mu, sigma))


mu = 0
variance = 5
sigma = math.sqrt(variance)
norm_2 = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(norm_2, stats.norm.pdf(norm_2, mu, sigma))
plt.ylabel('Density')
plt.xlabel('Value')
plt.show()


In [ ]:
## Binomial Distribution
from scipy.stats import binom

k = np.arange(11)
data_binom = binom.pmf(k, n = 10, p = 0.8)

ax_binom = sns.barplot(x = k, y = data_binom)

#ax_binom = plt.plot(data_binom)
ax_binom.set(xlabel='Successes', ylabel='Density')

ax_binom;



In [ ]:
## Poisson Distribution

from scipy.stats import poisson
k = np.arange(100, step = 5)
data_poisson = poisson.pmf(k = k, mu = 40)

ax_poisson = sns.barplot(x = k, y = data_poisson)

ax_poisson.set(xlabel='Count', ylabel='Density')
ax_poisson


In [ ]:
## Negative Binomial

from scipy.stats import nbinom
data_nbinom = nbinom.rvs(n = 5, p = 0.5, size=10000)
 
ax_nbinom = sns.distplot(data_nbinom, kde = False)
 
ax_nbinom.set(xlabel='Negative Binomial', ylabel='Frequency')
ax_nbinom


#### Exercise:

1. Experiment with the poisson distribution. What happens when you change mu?

2. In groups of 2-3, discuss some examples from your own work where you have a response variable that comes from a Binomial, Poisson, or Normal distribution?


In [ ]:
## Exercise 1: Experiment with the negative binomial distribution

from scipy.stats import nbinom
data_nbinom = nbinom.rvs(n = 5, p = 0.5, size=10000)
 
ax_nbinom = sns.distplot(data_nbinom, kde = False)
 
ax_nbinom.set(xlabel='Negative Binomial', ylabel='Frequency')
ax_nbinom
